In [121]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [75]:
%pip install -qU pypdf langchain_community

Note: you may need to restart the kernel to use updated packages.


In [77]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./Tomato Farming in Nigeria.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

13


In [79]:
print(docs[0].page_content[0:100])
print(docs[0].metadata)

Tomato Farming in Nigeria: A Complete Guide July 26, 2020  Share Tweet Share Tomato farming involves
{'source': './Tomato Farming in Nigeria.pdf', 'page': 0}


In [81]:
pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [123]:
# import getpass
# import os

# os.environ["GROQ_API_KEY"] = getpass.getpass()

# from langchain_groq import ChatGroq

# model = ChatGroq(model="llama3-8b-8192")
import os
from getpass import getpass

PPLX_API_KEY = getpass()
os.environ["PPLX_API_KEY"] = PPLX_API_KEY

chat = ChatPerplexity(temperature=0, model="pplx-70b-online")

 ········


In [125]:
%pip install langchain_chroma langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [127]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [152]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# system_prompt = (
#     "You are an assistant for question-answering tasks. "
#     "Use the following pieces of retrieved context to answer "
#     "the question. If you don't know the answer, say that you "
#     "don't know. Use three sentences maximum and keep the "
#     "answer concise."
#     "\n\n"
#     "{context}"
# )

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. Do not answer any question out of CONTEXT, Just say you can not provide an answer that is out of the scope using your own wordings"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm
                                                     , prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "What should be the soil ph to be used for tomato farming in Moroco?"})

results["answer"]

"I can't provide an answer that is out of scope. The given context is about tomato farming in Nigeria, not Morocco."

In [93]:
print(results["context"][0].page_content)

14. Tomato Harvest In tomato farming, harvesting typically commences 55-70 days after transplanting depending on the variety of tomato planted. Some varieties of tomatoes have thick fruits; these thick fruits last longer than the soft skin tomato varieties. It is advised that a farmer should have a way of preserving his tomato fruits after harvest. Tomatoes are usually harvested when they are red or yellow turning into red. Some tomato farmers harvest their tomatoes when green, I do not advise this as it may be difficult to turn the colour from green to red. People will not buy green tomatoes.. For those who have the financial capability, you can harvest your tomato fruits and process them into paste and puree on farm. Tomatoes can also be harvested, sun dried and milled into powder. 15. Post-Harvest and Marketing Post-harvest activities in tomato farming include sorting, packaging and storage of the harvested tomato fruits.


In [95]:
print(results["context"][0].metadata)

{'page': 9, 'source': './Tomato Farming in Nigeria.pdf'}
